#### Importing packages

In [79]:
import pandas as pd
from datetime import datetime, timedelta

import glob

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list(name='stock', colors=['red','white','green'])

In [80]:
from helper import *

#### Nifty 50 stocks analysis

In [81]:
## reading nifty50 stocks historical data
l = [pd.read_csv(filename) for filename in glob.glob("nifty50_stocks_historical_data_yahoo_finance/*.csv")]
df = pd.concat(l, axis=0)

## removing dates for which closing price is not populated
df = df[~df['close'].isnull()]

## processing date column
df['date'] = df["date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").date())
df["year"] = df["date"].apply(lambda x: x.year)

## filter: last 10 years
df = df[df['date'] >= datetime(2009, 12, 20).date()]

## sorting
df.sort_values('date', inplace=True, ignore_index=True)

## closing dates of each year
closing_dates = list(df.groupby(['year'])['date'].agg(['max'])['max'].values)

## filter: closing dates
df = df[df['date'].isin(closing_dates)]

In [82]:
analysis_df_1 = df.pivot_table(index='symbol', columns='year', values='close')

## yearly returns
for year in range(2010, 2020):
    analysis_df_1[str(year)] = analysis_df_1.apply(lambda x: absolute_return(x[year-1], x[year]), axis=1)

## CAGR
analysis_df_1["CAGR (1 Yr)"] = analysis_df_1["2019"]  
analysis_df_1["CAGR (2 Yr)"] = analysis_df_1.apply(lambda x: annualized_return(x[2017], x[2019], 2), axis=1)
analysis_df_1["CAGR (5 Yr)"] = analysis_df_1.apply(lambda x: annualized_return(x[2014], x[2019], 5), axis=1)
analysis_df_1["CAGR (10 Yr)"] = analysis_df_1.apply(lambda x: annualized_return(x[2009], x[2019], 10), axis=1)

## overall returns (CAGR (1 yr + 2 yr + 5 yr + 10 yr)/4)
analysis_df_1["overall_return"] = (analysis_df_1["CAGR (1 Yr)"] + analysis_df_1["CAGR (2 Yr)"] + analysis_df_1["CAGR (5 Yr)"] + analysis_df_1["CAGR (10 Yr)"])/4
    
## removing some columns
analysis_df_1 = analysis_df_1[[col_ for col_ in analysis_df_1.columns if isinstance(col_, str)]]

## sorting
analysis_df_1.sort_values(['overall_return', 'CAGR (5 Yr)', 'CAGR (2 Yr)', 'CAGR (1 Yr)'], ascending=[False, False, False, False], inplace=True)

## adding style to dataframe
analysis_df_1 = analysis_df_1.style.background_gradient(cmap=cmap, vmin=-200, vmax=200)

In [83]:
analysis_df_1

year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,CAGR (1 Yr),CAGR (2 Yr),CAGR (5 Yr),CAGR (10 Yr),overall_return
symbol,,,,,,,,,,,,,,,
BAJFINANCE,123.400000,-13.900000,123.100000,20.400000,120.300000,72.500000,40.100000,108.600000,50.600000,60.100000,60.100000,55.300000,64.800000,63.700000,60.975000
BAJAJFINSV,42.300000,-7.300000,114.300000,-17.800000,76.100000,52.000000,45.600000,80.700000,23.800000,45.000000,45.000000,34.000000,48.300000,40.200000,41.875000
KOTAKBANK,13.600000,-5.100000,51.000000,12.000000,73.600000,13.900000,-0.000000,40.400000,24.400000,34.100000,34.100000,29.100000,21.700000,23.800000,27.175000
NESTLEIND,52.900000,7.400000,21.900000,5.900000,20.700000,-8.700000,3.400000,30.600000,40.800000,33.400000,33.400000,37.100000,18.300000,19.500000,27.075000
ICICIBANK,38.700000,-40.200000,66.300000,-3.500000,60.700000,-26.000000,-2.300000,35.300000,14.700000,49.600000,49.600000,31.000000,10.900000,13.600000,26.275000
TITAN,162.600000,-4.900000,66.300000,-19.300000,66.300000,-9.000000,-5.900000,162.600000,8.500000,27.600000,27.600000,17.700000,25.500000,33.000000,25.950000
RELIANCE,3.900000,-34.500000,21.200000,6.600000,-0.500000,13.900000,6.700000,70.200000,21.700000,35.000000,35.000000,28.200000,27.700000,11.500000,25.600000
ASIANPAINT,65.800000,-9.900000,71.000000,10.500000,53.500000,17.400000,0.800000,30.000000,18.500000,30.000000,30.000000,24.100000,18.900000,26.200000,24.800000
HDFC,42.100000,-10.500000,27.100000,-1.300000,0.000000,54.500000,-0.100000,35.500000,15.100000,22.600000,22.600000,18.800000,24.100000,16.800000,20.575000


#### References

- https://www.moneycontrol.com/news/business/markets/these-12-nifty-stocks-gave-double-digit-cagr-in-10-years-are-they-good-for-your-portfolio-4847181.html
- https://getmoneyrich.com/historical-returns-given-top-indian-stocks/
- https://www.marketwatch.com/story/these-are-the-20-best-performing-stocks-of-the-past-decade-and-some-of-them-will-surprise-you-2019-12-09

In [ ]:
#no